# 8 - Run All Optimisations: Memory Capacity with Noise (Generates Data for Figure 15)

This notebook re-generates data for the section from *Section 9.4: Combining the Optimisation Approaches*. If you would like to just plot the pre-generated paper results, use the Jupyter notebook `9 - Experiment_B_all_opts - Plot Fig 15.ipynb`. 


**To just plot the results:** 
If you would like to just plot the pre-generated paper results, there is no need to re-generate the results as they are cached in `examples/expermiment_results/experiment_B_all_opts`. In this case, simply use the Jupyter notebook `9 - Experiment_B_all_opts - Plot Fig 15`. 

**Where the data will be saved:** The results from this test willl be saved in the existing Excel files in the `examples/expermiment_results/experiment_B_all_opts` folder and overwrite the sheet named `latest_data`. The data generated for the paper is on the `paper_data` sheet, which will not be overwritten. You can select the data to plot (the latest or the paper data) by selecting the correct sheet name in `9 - Experiment_B_all_opts - Plot Fig 15`. 


*Note that the full set of data took many hours to run on a high performance computing platform [1] and the data was collected in batches.*

The following experiment is used by default:

<table>
<tr><td>

| Problem B (varying memories)   |          |
| :----------- | :------  | 
| $f$          | 2,000      | 
| $m$          | Varying | 
| $s_m$        | 0.1     | 
| $s_n$        | Varying      | 
</td><td>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
</td><td>

| Fully Optimised Network |          |
| :--------------------| :------  | 
| $h$                  | 2        | 
| $\theta$             | Optimised      | 
| $s^+_{f\_h}$         | 0.1      | 
| $\sigma_{h\_pre+}$   | `FixedNumber` | 
| $s^+_{h\_f}$         | 1 or 0.5    | 
| $s^-_{h\_f}$         | 1 or 0.5 or 0   | 
| $\sigma_{f\_pre+}$, $\sigma_{f\_pre-}$  |  `FixedNumber`       | 
| $e$                  | 3        | 

</td></tr> </table>

The accuracy of three different networks are studied in a noise environment as the number of memories $m$ varies:

* Fully Optimised Network - Full modulation $s^+_{h\_f} = 1$ and $s^-_{h\_f} = 1$
* Fully Optimised Network - Partial modulation $s^+_{h\_f} = 0.5$ and $s^-_{h\_f} = 0.5$
* Fully Optimised Network - Full excitatory modulation $s^+_{h\_f} = 1$ and no inhibitory modulation $s^-_{h\_f} = 0$

[1] The IRIDIS High Performance Computing Facility at the University of Southampton.

## Dependencies and problem/network definition

Import the dependencies and define the tests to run. 

### Dependencies

In [1]:
import os
import sys
sys.path.append('../src')


In [2]:
output_base_dir = '.'+os.sep+'experiment_results'
test_name = 'experiment_B_all_opts'

## Set up the test suite details

First set up the baseline configuration.

In [3]:
# Default values for the problem and network
f = 2000
s_m = 200/f
problem_space_default = {'f': f,
                         'm': 1000,
                         's_m': s_m,
                         's_n': 0.1,
                         's_is_prob': False}

network_default = {'f': f,
                   'h': 2,
                   'f_h_sparsity': 0.1,
                   'h_f_sparsity_e': 1,
                   'h_f_sparsity_i': 0.5,
                   'e': 3,
                   'h_thresh': -1,    # Indicates optimise
                   'f_h_conn_type': 'FixedNumberPre',
                   'h_f_conn_type': 'FixedNumberPre',
                   'debug': False
                   }

### Define the variable that will change (the x-axis on the final plots)

In this case, the variable that is changed is the number of memories `m`.

Note that the number of memories in the paper was considerably higher (2,000,000) then in the following cell, but running with this numnber of memories is computationally expensive using this matrix implementation on conventional hardware (see the note in the first cell relating to the use of high performance compute facilities).

Update this following cell to define the tests that you would like to run.

In [4]:
vary_memories = [2000, 6000]

### 
Define the networks to be tested. 

Each network will be tested multiple times for each of the variable options.

The `problem_vary` and `network_static` dictionary keys indicate that the variation betweeen the tests (for each network) is the problem space - specifically in this case, the number of memories `m`

There are 3 network varieties:
* `fully_optimised_full_mod`: Fully Optimised Network - Full modulation $s^+_{h\_f} = 1$ and $s^-_{h\_f} = 1$
* `fully_optimised_partial_mod`: Fully Optimised Network - Partial modulation $s^+_{h\_f} = 0.5$ and $s^-_{h\_f} = 0.5$
* `fully_optimised_no_inhib`: Fully Optimised Network - Full excitatory modulation $s^+_{h\_f} = 1$ and no inhibitory modulation $s^-_{h\_f} = 0$

In [5]:
import copy

fully_optimised_full_mod ={
            'problem_vary': problem_space_default.copy(),
            'network_static': network_default.copy(),
            'plot_params':
                {'variable_column': 'm',
                 'variable_column_name': 'Number of memories',
                 'variable_is_integer': False}
         }

# Override with the variable parameter. Specifying an array for this parameter will
# indicate to the test runner that there are multiple scenarios to be tested.
fully_optimised_full_mod['problem_vary'][fully_optimised_full_mod['plot_params']['variable_column']] = vary_memories

fully_optimised_partial_mod = copy.deepcopy(fully_optimised_full_mod)
fully_optimised_partial_mod['network_static']['h_f_sparsity_e'] = 0.5
fully_optimised_partial_mod['network_static']['h_f_sparsity_i'] = 0.5

fully_optimised_no_inhib = copy.deepcopy(fully_optimised_full_mod)
fully_optimised_no_inhib['network_static']['h_f_sparsity_i'] = 0


Put all the network descriptions into a single dictionary for the test runner.

In [6]:
all_tests = {'fully_optimised_full_mod': fully_optimised_full_mod,
            'fully_optimised_partial_mod': fully_optimised_partial_mod,
            'fully_optimised_no_inhib': fully_optimised_no_inhib}

## Run the tests

`num_tests` is the number of networks for each type of test that will be created. Despite having the same network definition, each will vary due to randomness in the data and network connections.

`simulations_per_test` is the number of recalls that will be run on each test network.

The total number of tests that will be run is: `number of test networks (1) * len(vary_m) * num_tests * simulations_per_test`

This test is classified as `vary_problem` because the network remains static for each optimisation test, but the problem is varied. 

In [7]:
num_tests = 2
sims_per_test = 2
output_dir = output_base_dir + os.sep + test_name

In [8]:
from simulation_scripts.test_suite_runner import TestSuiteRunner

tsr = TestSuiteRunner(network_default=network_default,
                      problem_space_default=problem_space_default,
                      output_dir=output_dir)


tsr.run_vary_problem_space_test_suite(all_tests = all_tests,
                                      num_tests = num_tests,
                                      sims_per_test = sims_per_test)


Running test set: fully_optimised_full_mod defined as: 
    Static network params:    {'f': 2000, 'h': 2, 'f_h_sparsity': 0.1, 'h_f_sparsity_e': 1, 'h_f_sparsity_i': 0.5, 'e': 3, 'h_thresh': -1, 'f_h_conn_type': 'FixedNumberPre', 'h_f_conn_type': 'FixedNumberPre', 'debug': False}
    Varying problem space:    {'f': 2000, 'm': [2000, 6000], 's_m': 0.1, 's_n': 0.1, 's_is_prob': False}

Number of tests per scenario: 2 
Number of simulations per test: 2 

--------------------------------------------------------------------------------------------
Simulation 1 of 2 for test 1 of 2

======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   2000
      Number of simulations:                2
      Signal length:                        200
      Bits to flip for noise:               200
Establishing hidden neuron threshold for the following test:
    net_param_f:             2000
    net_param_h:             2
    ne

In [9]:
print('Experiment complete, data stored in:\n')
print('  {:80s}    {:10s}\n'.format('File', 'Sheet'))

for test_file, _ in all_tests.items():
    excel_file = output_base_dir + os.sep + test_name + os.sep + test_file + '.xlsx'
    print('  {:80s}    {:10s}'.format(excel_file, 'latest_data'))

Experiment complete, data stored in:

  File                                                                                Sheet     

  ./experiment_results/experiment_B_all_opts/fully_optimised_full_mod.xlsx            latest_data
  ./experiment_results/experiment_B_all_opts/fully_optimised_partial_mod.xlsx         latest_data
  ./experiment_results/experiment_B_all_opts/fully_optimised_no_inhib.xlsx            latest_data


## Next steps: Plotting the results

Use the Jupyter notebook `9 - Experiment_B_all_opts - Plot Fig 15.ipynb` to plot the results.

You will need to update the `data_sheet` variable to `latest_data` as described in the notebook, else the pre-generated paper data will be plotted.